[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oreilly-japan/RecommenderSystems/blob/main/chapter5/colab/Popularity.ipynb)

# 人気度順推薦
## 人気度の定義
* 今回は評価値が高いものを人気が髙い映画とする
* 人気度の定義は「クリック数が多いもの」「購入が多いもの」「評価値が髙いもの」など複数あり、自社サービスに最も適した定義を利用


In [ ]:
# Colab用のnotebookです。このnotebook1枚でデータのダウンロードから、レコメンドまで完結するようになっています。（予測評価は含めていません。）
# MovieLensデータがまだダウンロードされてなければこのセルを実行して、ダウンロードしてください
# MovieLensデータの分析は、data_download.ipynbをご参照ください

# データのダウンロードと解凍
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data
!unzip -n ../data/ml-10m.zip -d ../data/

In [1]:
# Movielensのデータの読み込み（データ量が多いため、読み込みに時間がかかる場合があります）
import pandas as pd

# movieIDとタイトル名のみ使用
m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv('../data/ml-10M100K/movies.dat', names=m_cols, sep='::' , encoding='latin-1', engine='python')

# genreをlist形式で保持する
movies['genre'] = movies.genre.apply(lambda x:x.split('|'))


# ユーザが付与した映画のタグ情報の読み込み
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv('../data/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')

# tagを小文字にする
user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()


# tagを映画ごとにlist形式で保持する
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list})

# タグ情報を結合する
movies = movies.merge(movie_tags, on='movie_id', how='left')

# 評価値データの読み込み
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/ml-10M100K/ratings.dat', names=r_cols, sep='::', engine='python')


# データ量が多いため、ユーザー数を1000に絞って、試していく
valid_user_ids = sorted(ratings.user_id.unique())[:1000]
ratings = ratings[ratings["user_id"].isin(valid_user_ids)]


# 映画のデータと評価のデータを結合する
movielens = ratings.merge(movies, on='movie_id')

print(f'unique_users={len(movielens.user_id.unique())}, unique_movies={len(movielens.movie_id.unique())}')

# 学習用とテスト用にデータを分割する
# 各ユーザの直近の５件の映画を評価用に使い、それ以外を学習用とする
# まずは、それぞれのユーザが評価した映画の順序を計算する
# 直近付与した映画から順番を付与していく(1始まり)

movielens['timestamp_rank'] = movielens.groupby(
    'user_id')['timestamp'].rank(ascending=False, method='first')
movielens_train = movielens[movielens['timestamp_rank'] > 5]
movielens_test = movielens[movielens['timestamp_rank']<= 5]

unique_users=1000, unique_movies=6736


In [2]:
# 評価数の閾値
minimum_num_rating = 200


In [15]:
import numpy as np

# 評価値の平均が髙い映画の確認
# 評価数が１件の映画が多数上位にきている
movie_stats = movielens_train.groupby(['movie_id', 'title']).agg({'rating': [np.size, np.mean]})
movie_stats.sort_values(by=('rating', 'mean'), ascending=False).head()

rating     
                                                              size mean
movie_id title                                                         
4095     Cry Freedom (1987)                                      1  5.0
7227     Trouble with Angels, The (1966)                         1  5.0
27255    Wind Will Carry Us, The (Bad ma ra khahad bord)...      1  5.0
4453     Michael Jordan to the Max (2000)                        2  5.0
3415     Mirror, The (Zerkalo) (1975)                            1  5.0

In [18]:
# しきい値を導入して、評価数が少ない映画を取り除く
# ショーシャンクの空にや七人の侍などの見慣れた映画が上位にきている
movie_stats = movielens_train.groupby(['movie_id', 'title']).agg({'rating': [np.size, np.mean]})
atleast_flg = movie_stats['rating']['size'] >= 100
movies_sorted_by_rating = movie_stats[atleast_flg].sort_values(by=('rating', 'mean'), ascending=False)
movies_sorted_by_rating.head()

rating          
                                                       size      mean
movie_id title                                                       
318      Shawshank Redemption, The (1994)               424  4.491745
50       Usual Suspects, The (1995)                     334  4.459581
912      Casablanca (1942)                              163  4.444785
904      Rear Window (1954)                             129  4.441860
2019     Seven Samurai (Shichinin no samurai) (1954)    104  4.408654

In [19]:
import numpy as np 
# 各アイテムごとの平均の評価値を計算し、その平均評価値を予測値として利用する
movie_rating_average = movielens_train.groupby("movie_id").agg({"rating": np.mean})
# テストデータに予測値を格納する。テストデータのみに存在するアイテムの予測評価値は０とする
movie_rating_predict = movielens_test.merge(
    movie_rating_average, on="movie_id", how="left", suffixes=("_test", "_pred")
).fillna(0)

In [20]:
from collections import defaultdict

# 各ユーザに対するおすすめ映画は、そのユーザがまだ評価していない映画の中から評価値が高いもの10作品とする
# ただし、評価件数が少ないとノイズが大きいため、minimum_num_rating件以上評価がある映画に絞る
pred_user2items = defaultdict(list)
user_watched_movies = movielens_train.groupby("user_id").agg({"movie_id": list})["movie_id"].to_dict()
movie_stats = movielens_train.groupby("movie_id").agg({"rating": [np.size, np.mean]})
atleast_flg = movie_stats["rating"]["size"] >= minimum_num_rating
movies_sorted_by_rating = (
    movie_stats[atleast_flg].sort_values(by=("rating", "mean"), ascending=False).index.tolist()
)

for user_id in movielens_train.user_id.unique():
    for movie_id in movies_sorted_by_rating:
        if movie_id not in user_watched_movies[user_id]:
            pred_user2items[user_id].append(movie_id)
        if len(pred_user2items[user_id]) == 10:
            break
pred_user2items

defaultdict(list,
            {139: [50, 858, 1193, 1214, 1200, 1291, 457, 150, 2396, 34],
             149: [318, 527, 260, 1193, 593, 2571, 1136, 2028, 1197, 2762],
             182: [260, 541, 1198, 1214, 1200, 47, 1036, 1291, 1210, 1240],
             215: [527, 590, 1073, 539, 253, 2683, 231],
             281: [858, 260, 1193, 2571, 2959, 1196, 2858, 1198, 1617, 1136],
             326: [318, 858, 527, 260, 1193, 2571, 541, 2959, 1196, 2858],
             351: [541, 2959, 2028, 2762, 1200, 3578, 47, 1291, 1240, 589],
             357: [318, 50, 858, 260, 1193, 593, 541, 2959, 1196, 2858],
             426: [1073, 141, 329],
             456: [858, 527, 260, 1193, 2571, 541, 2959, 1196, 2858, 1198],
             459: [318, 50, 858, 1193, 1198, 1617, 1197, 608, 457, 1265],
             494: [858, 260, 1193, 2571, 541, 2959, 1196, 2858, 1198, 1617],
             517: [858, 527, 260, 1193, 593, 2571, 541, 2959, 1196, 2858],
             524: [527, 260, 1193, 593, 1196, 1198, 1617, 11

In [11]:
# user_id=2のユーザーが学習データで評価を付けた映画一覧
movielens_train[movielens_train.user_id==2]

,user_id,movie_id,rating,timestamp,title,genre,tag,timestamp_rank
4732,2,110,5.0,868245777,Braveheart (1995),"[Action, Drama, War]","[bullshit history, medieval, bloodshed, hero, ...",8.0
5246,2,260,5.0,868244562,Star Wars: Episode IV - A New Hope (a.k.a. Sta...,"[Action, Adventure, Sci-Fi]","[desert, quotable, lucas, gfei own it, seen mo...",17.0
5798,2,590,5.0,868245608,Dances with Wolves (1990),"[Adventure, Drama, Western]","[afi 100, lame, native, biopic, american india...",11.0
6150,2,648,2.0,868244699,Mission: Impossible (1996),"[Action, Adventure, Mystery, Thriller]","[confusing, confusing plot, memorable sequence...",12.0
6531,2,733,3.0,868244562,"Rock, The (1996)","[Action, Adventure, Thriller]","[gfei own it, alcatraz, nicolas cage, sean con...",18.0
6813,2,736,3.0,868244698,Twister (1996),"[Action, Adventure, Romance, Thriller]","[disaster, disaster, storm, bill paxton, helen...",13.0
7113,2,780,3.0,868244698,Independence Day (a.k.a. ID4) (1996),"[Action, Adventure, Sci-Fi, War]","[action, alien invasion, aliens, will smith, a...",14.0
7506,2,786,3.0,868244562,Eraser (1996),"[Action, Drama, Thriller]","[arnold schwarzenegger, action, arnold, arnold...",19.0
7661,2,802,2.0,868244603,Phenomenon (1996),"[Drama, Romance]","[interesting concept, own, john travolta, john...",15.0
7779,2,858,2.0,868245645,"Godfather, The (1972)","[Crime, Drama]","[oscar (best picture), marlon brando, classic,...",9.0


In [12]:
pred_user2items[2]

[318, 50, 527, 1193, 593, 2571, 541, 2959, 1196, 2858]

In [13]:
# user_id=2に対するおすすめ(318, 50, 527)
movies[movies.movie_id.isin([318, 50, 527])]

,movie_id,title,genre,tag
49,50,"Usual Suspects, The (1995)","[Crime, Mystery, Thriller]","[kevin spacey, ensemble cast, complicated, mus..."
315,318,"Shawshank Redemption, The (1994)",[Drama],"[based on a short story, directorial debut, fr..."
523,527,Schindler's List (1993),"[Drama, War]","[speilberg, drama, holocaust, steven spielberg..."
